# w261 Final Project - Clickthrough Rate Prediction


[Your team number (from the spreadsheet)]   
[Your team names]   
Summer 2019, section [Your section numbers>]   

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts

In [1]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "Final Project W261"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [4]:
spark

In [5]:
sc = spark.sparkContext

In [8]:
dftoy_tiny = spark.read.load("toy_train.parquet")

In [9]:
#create spark dataframe for train - Set toyDF to read train_80 or use dftoy sample
#toyDF = spark.read.load("train_80.parquet")
toyDF = dftoy_tiny

#print Schema
toyDF.printSchema()

#convert all Label, Integer columns to Ints
from pyspark.sql.types import IntegerType
toyDF = toyDF.withColumn("Label", toyDF["Label"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-1", toyDF["I-1"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-2", toyDF["I-2"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-3", toyDF["I-3"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-4", toyDF["I-4"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-5", toyDF["I-5"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-6", toyDF["I-6"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-7", toyDF["I-7"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-8", toyDF["I-8"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-9", toyDF["I-9"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-10", toyDF["I-10"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-11", toyDF["I-11"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-12", toyDF["I-12"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-13", toyDF["I-13"].cast(IntegerType()))

#fill all Integer columns `NA` to `0'
toyDF = toyDF.fillna(0)

# Replace all categorical string `empty` to `null`
toyDF = toyDF.replace('', 'null')

# convert all `hex` code to `int`
from pyspark.sql.functions import conv
toyDF = toyDF.withColumn('C-1', conv('C-1', 16, 10))
toyDF = toyDF.withColumn('C-2', conv('C-2', 16, 10))
toyDF = toyDF.withColumn('C-3', conv('C-3', 16, 10))
toyDF = toyDF.withColumn('C-4', conv('C-4', 16, 10))
toyDF = toyDF.withColumn('C-5', conv('C-5', 16, 10))
toyDF = toyDF.withColumn('C-6', conv('C-6', 16, 10))
toyDF = toyDF.withColumn('C-7', conv('C-7', 16, 10))
toyDF = toyDF.withColumn('C-8', conv('C-8', 16, 10))
toyDF = toyDF.withColumn('C-9', conv('C-9', 16, 10))
toyDF = toyDF.withColumn('C-10', conv('C-10', 16, 10))
toyDF = toyDF.withColumn('C-11', conv('C-11', 16, 10))
toyDF = toyDF.withColumn('C-12', conv('C-12', 16, 10))
toyDF = toyDF.withColumn('C-13', conv('C-13', 16, 10))
toyDF = toyDF.withColumn('C-14', conv('C-14', 16, 10))
toyDF = toyDF.withColumn('C-15', conv('C-15', 16, 10))
toyDF = toyDF.withColumn('C-16', conv('C-16', 16, 10))
toyDF = toyDF.withColumn('C-17', conv('C-17', 16, 10))
toyDF = toyDF.withColumn('C-18', conv('C-18', 16, 10))
toyDF = toyDF.withColumn('C-19', conv('C-19', 16, 10))
toyDF = toyDF.withColumn('C-20', conv('C-20', 16, 10))
toyDF = toyDF.withColumn('C-21', conv('C-21', 16, 10))
toyDF = toyDF.withColumn('C-22', conv('C-22', 16, 10))
toyDF = toyDF.withColumn('C-23', conv('C-23', 16, 10))
toyDF = toyDF.withColumn('C-24', conv('C-24', 16, 10))
toyDF = toyDF.withColumn('C-25', conv('C-25', 16, 10))
toyDF = toyDF.withColumn('C-26', conv('C-26', 16, 10))

#check if its good
toyDF.show(3)

# convert all categorial feature columns from string to long type
from pyspark.sql.types import LongType
toyDF = toyDF.withColumn("C-1", toyDF["C-1"].cast(LongType()))
toyDF = toyDF.withColumn("C-2", toyDF["C-2"].cast(LongType()))
toyDF = toyDF.withColumn("C-3", toyDF["C-3"].cast(LongType()))
toyDF = toyDF.withColumn("C-4", toyDF["C-4"].cast(LongType()))
toyDF = toyDF.withColumn("C-5", toyDF["C-5"].cast(LongType()))
toyDF = toyDF.withColumn("C-6", toyDF["C-6"].cast(LongType()))
toyDF = toyDF.withColumn("C-7", toyDF["C-7"].cast(LongType()))
toyDF = toyDF.withColumn("C-8", toyDF["C-8"].cast(LongType()))
toyDF = toyDF.withColumn("C-9", toyDF["C-9"].cast(LongType()))
toyDF = toyDF.withColumn("C-10", toyDF["C-10"].cast(LongType()))
toyDF = toyDF.withColumn("C-11", toyDF["C-11"].cast(LongType()))
toyDF = toyDF.withColumn("C-12", toyDF["C-12"].cast(LongType()))
toyDF = toyDF.withColumn("C-13", toyDF["C-13"].cast(LongType()))
toyDF = toyDF.withColumn("C-14", toyDF["C-14"].cast(LongType()))
toyDF = toyDF.withColumn("C-15", toyDF["C-15"].cast(LongType()))
toyDF = toyDF.withColumn("C-16", toyDF["C-16"].cast(LongType()))
toyDF = toyDF.withColumn("C-17", toyDF["C-17"].cast(LongType()))
toyDF = toyDF.withColumn("C-18", toyDF["C-18"].cast(LongType()))
toyDF = toyDF.withColumn("C-19", toyDF["C-19"].cast(LongType()))
toyDF = toyDF.withColumn("C-20", toyDF["C-20"].cast(LongType()))
toyDF = toyDF.withColumn("C-21", toyDF["C-21"].cast(LongType()))
toyDF = toyDF.withColumn("C-22", toyDF["C-22"].cast(LongType()))
toyDF = toyDF.withColumn("C-23", toyDF["C-23"].cast(LongType()))
toyDF = toyDF.withColumn("C-24", toyDF["C-24"].cast(LongType()))
toyDF = toyDF.withColumn("C-25", toyDF["C-25"].cast(LongType()))
toyDF = toyDF.withColumn("C-26", toyDF["C-26"].cast(LongType()))

# verify the schema
toyDF.printSchema()

# verify the data
toyDF.show(3)

trainDF = toyDF

root
 |-- Label: string (nullable = true)
 |-- I-1: string (nullable = true)
 |-- I-2: string (nullable = true)
 |-- I-3: string (nullable = true)
 |-- I-4: string (nullable = true)
 |-- I-5: string (nullable = true)
 |-- I-6: string (nullable = true)
 |-- I-7: string (nullable = true)
 |-- I-8: string (nullable = true)
 |-- I-9: string (nullable = true)
 |-- I-10: string (nullable = true)
 |-- I-11: string (nullable = true)
 |-- I-12: string (nullable = true)
 |-- I-13: string (nullable = true)
 |-- C-1: string (nullable = true)
 |-- C-2: string (nullable = true)
 |-- C-3: string (nullable = true)
 |-- C-4: string (nullable = true)
 |-- C-5: string (nullable = true)
 |-- C-6: string (nullable = true)
 |-- C-7: string (nullable = true)
 |-- C-8: string (nullable = true)
 |-- C-9: string (nullable = true)
 |-- C-10: string (nullable = true)
 |-- C-11: string (nullable = true)
 |-- C-12: string (nullable = true)
 |-- C-13: string (nullable = true)
 |-- C-14: string (nullable = true)
 |--

In [10]:
#create spark dataframe for validation - Set toyDF to read validation_20 or use dftoy sample
#toyDF = spark.read.load("validation_20.parquet")
toyDF = dftoy_tiny

#print Schema
toyDF.printSchema()

#convert all Label, Integer columns to Ints
from pyspark.sql.types import IntegerType
toyDF = toyDF.withColumn("Label", toyDF["Label"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-1", toyDF["I-1"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-2", toyDF["I-2"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-3", toyDF["I-3"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-4", toyDF["I-4"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-5", toyDF["I-5"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-6", toyDF["I-6"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-7", toyDF["I-7"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-8", toyDF["I-8"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-9", toyDF["I-9"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-10", toyDF["I-10"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-11", toyDF["I-11"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-12", toyDF["I-12"].cast(IntegerType()))
toyDF = toyDF.withColumn("I-13", toyDF["I-13"].cast(IntegerType()))

#fill all Integer columns `NA` to `0'
toyDF = toyDF.fillna(0)

# Replace all categorical string `empty` to `null`
toyDF = toyDF.replace('', 'null')

# convert all `hex` code to `int`
from pyspark.sql.functions import conv
toyDF = toyDF.withColumn('C-1', conv('C-1', 16, 10))
toyDF = toyDF.withColumn('C-2', conv('C-2', 16, 10))
toyDF = toyDF.withColumn('C-3', conv('C-3', 16, 10))
toyDF = toyDF.withColumn('C-4', conv('C-4', 16, 10))
toyDF = toyDF.withColumn('C-5', conv('C-5', 16, 10))
toyDF = toyDF.withColumn('C-6', conv('C-6', 16, 10))
toyDF = toyDF.withColumn('C-7', conv('C-7', 16, 10))
toyDF = toyDF.withColumn('C-8', conv('C-8', 16, 10))
toyDF = toyDF.withColumn('C-9', conv('C-9', 16, 10))
toyDF = toyDF.withColumn('C-10', conv('C-10', 16, 10))
toyDF = toyDF.withColumn('C-11', conv('C-11', 16, 10))
toyDF = toyDF.withColumn('C-12', conv('C-12', 16, 10))
toyDF = toyDF.withColumn('C-13', conv('C-13', 16, 10))
toyDF = toyDF.withColumn('C-14', conv('C-14', 16, 10))
toyDF = toyDF.withColumn('C-15', conv('C-15', 16, 10))
toyDF = toyDF.withColumn('C-16', conv('C-16', 16, 10))
toyDF = toyDF.withColumn('C-17', conv('C-17', 16, 10))
toyDF = toyDF.withColumn('C-18', conv('C-18', 16, 10))
toyDF = toyDF.withColumn('C-19', conv('C-19', 16, 10))
toyDF = toyDF.withColumn('C-20', conv('C-20', 16, 10))
toyDF = toyDF.withColumn('C-21', conv('C-21', 16, 10))
toyDF = toyDF.withColumn('C-22', conv('C-22', 16, 10))
toyDF = toyDF.withColumn('C-23', conv('C-23', 16, 10))
toyDF = toyDF.withColumn('C-24', conv('C-24', 16, 10))
toyDF = toyDF.withColumn('C-25', conv('C-25', 16, 10))
toyDF = toyDF.withColumn('C-26', conv('C-26', 16, 10))

#check if its good
toyDF.show(3)

# convert all categorial feature columns from string to long type
from pyspark.sql.types import LongType
toyDF = toyDF.withColumn("C-1", toyDF["C-1"].cast(LongType()))
toyDF = toyDF.withColumn("C-2", toyDF["C-2"].cast(LongType()))
toyDF = toyDF.withColumn("C-3", toyDF["C-3"].cast(LongType()))
toyDF = toyDF.withColumn("C-4", toyDF["C-4"].cast(LongType()))
toyDF = toyDF.withColumn("C-5", toyDF["C-5"].cast(LongType()))
toyDF = toyDF.withColumn("C-6", toyDF["C-6"].cast(LongType()))
toyDF = toyDF.withColumn("C-7", toyDF["C-7"].cast(LongType()))
toyDF = toyDF.withColumn("C-8", toyDF["C-8"].cast(LongType()))
toyDF = toyDF.withColumn("C-9", toyDF["C-9"].cast(LongType()))
toyDF = toyDF.withColumn("C-10", toyDF["C-10"].cast(LongType()))
toyDF = toyDF.withColumn("C-11", toyDF["C-11"].cast(LongType()))
toyDF = toyDF.withColumn("C-12", toyDF["C-12"].cast(LongType()))
toyDF = toyDF.withColumn("C-13", toyDF["C-13"].cast(LongType()))
toyDF = toyDF.withColumn("C-14", toyDF["C-14"].cast(LongType()))
toyDF = toyDF.withColumn("C-15", toyDF["C-15"].cast(LongType()))
toyDF = toyDF.withColumn("C-16", toyDF["C-16"].cast(LongType()))
toyDF = toyDF.withColumn("C-17", toyDF["C-17"].cast(LongType()))
toyDF = toyDF.withColumn("C-18", toyDF["C-18"].cast(LongType()))
toyDF = toyDF.withColumn("C-19", toyDF["C-19"].cast(LongType()))
toyDF = toyDF.withColumn("C-20", toyDF["C-20"].cast(LongType()))
toyDF = toyDF.withColumn("C-21", toyDF["C-21"].cast(LongType()))
toyDF = toyDF.withColumn("C-22", toyDF["C-22"].cast(LongType()))
toyDF = toyDF.withColumn("C-23", toyDF["C-23"].cast(LongType()))
toyDF = toyDF.withColumn("C-24", toyDF["C-24"].cast(LongType()))
toyDF = toyDF.withColumn("C-25", toyDF["C-25"].cast(LongType()))
toyDF = toyDF.withColumn("C-26", toyDF["C-26"].cast(LongType()))

# verify the schema
toyDF.printSchema()

# verify the data
toyDF.show(3)

validationDF = toyDF

root
 |-- Label: string (nullable = true)
 |-- I-1: string (nullable = true)
 |-- I-2: string (nullable = true)
 |-- I-3: string (nullable = true)
 |-- I-4: string (nullable = true)
 |-- I-5: string (nullable = true)
 |-- I-6: string (nullable = true)
 |-- I-7: string (nullable = true)
 |-- I-8: string (nullable = true)
 |-- I-9: string (nullable = true)
 |-- I-10: string (nullable = true)
 |-- I-11: string (nullable = true)
 |-- I-12: string (nullable = true)
 |-- I-13: string (nullable = true)
 |-- C-1: string (nullable = true)
 |-- C-2: string (nullable = true)
 |-- C-3: string (nullable = true)
 |-- C-4: string (nullable = true)
 |-- C-5: string (nullable = true)
 |-- C-6: string (nullable = true)
 |-- C-7: string (nullable = true)
 |-- C-8: string (nullable = true)
 |-- C-9: string (nullable = true)
 |-- C-10: string (nullable = true)
 |-- C-11: string (nullable = true)
 |-- C-12: string (nullable = true)
 |-- C-13: string (nullable = true)
 |-- C-14: string (nullable = true)
 |--

In [11]:
# Build trainDF and validationDF with vector inputs

from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler


assemblerInputs = ['I-1','I-2','I-3','I-4','I-5','I-6','I-7','I-8','I-9',
                                        'I-10','I-11','I-12','I-13','C-1','C-2','C-3','C-4','C-5','C-6',
                                        'C-7','C-8','C-9','C-10','C-11','C-12','C-13','C-14','C-15',
                                        'C-16','C-17','C-18','C-19','C-20','C-21','C-22','C-23','C-24',
                                        'C-25','C-26']

# Drop I-12 and C-22 with 76% missing values
ignore = ['I-12', 'C-22', 'Label']

# Build vector for decision tree for train data set
assembler = VectorAssembler(
     inputCols=[x for x in trainDF.columns if x not in ignore],
     outputCol='features')

# Transform the data for train data set
train_output = assembler.transform(trainDF)
train_output.select("Label", "features").show(truncate=False)

# Build vector for decision dree for validation data set
assembler = VectorAssembler(
     inputCols=[x for x in validationDF.columns if x not in ignore],
     outputCol='features')

# Transform the data for validation data set
validation_output = assembler.transform(validationDF)
validation_output.select("Label", "features").show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Label|features                                                                                                                                                                                                                                                                                                                                                                                                    |
+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
# See if it matches test
validationDF.show(20)

+-----+---+---+---+---+-----+---+---+---+----+----+----+----+----+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+----------+----------+----------+----------+----------+----------+----------+
|Label|I-1|I-2|I-3|I-4|  I-5|I-6|I-7|I-8| I-9|I-10|I-11|I-12|I-13|       C-1|       C-2|       C-3|       C-4|       C-5|       C-6|       C-7|       C-8|       C-9|      C-10|      C-11|      C-12|      C-13|      C-14|      C-15|      C-16|      C-17|      C-18|     C-19|      C-20|      C-21|      C-22|      C-23|      C-24|      C-25|      C-26|
+-----+---+---+---+---+-----+---+---+---+----+----+----+----+----+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+----------+----------+

In [13]:
# Check the data sizes

train = train_output
test = validation_output

print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

test.select("Label", "features").show(10, truncate=False)

Training Dataset Count: 100
Test Dataset Count: 100
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Label|features                                                                                                                                                                                                                                                                                                                                                                                                    |
+-----+-----------------------------------------------------------------------------------------------------------------------------------

In [38]:
### Train and test the model to make predicitions with decision tree

from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'Label', maxDepth = 10)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select('Label', 'rawPrediction', 'prediction', 'probability').show(25)

+-----+-------------+----------+-----------+
|Label|rawPrediction|prediction|probability|
+-----+-------------+----------+-----------+
|    0|   [25.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [32.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [32.0,0.0]|       0.0|  [1.0,0.0]|
|    0|    [8.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [32.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [32.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [25.0,0.0]|       0.0|  [1.0,0.0]|
|    1|    [0.0,2.0]|       1.0|  [0.0,1.0]|
|    0|   [25.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [25.0,0.0]|       0.0|  [1.0,0.0]|
|    0|    [9.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [25.0,0.0]|       0.0|  [1.0,0.0]|
|    1|    [0.0,2.0]|       1.0|  [0.0,1.0]|
|    1|    [0.0,1.0]|       1.0|  [0.0,1.0]|
|    0|    [9.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [25.0,0.0]|       0.0|  [1.0,0.0]|
|    1|    [0.0,1.0]|       1.0|  [0.0,1.0]|
|    0|    [3.0,0.0]|       0.0|  [1.0,0.0]|
|    0|    [3.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   

In [71]:
### Train and test a Random Forest Classifier to make predicitions

from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'Label', maxDepth = 10, numTrees=20)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('Label', 'rawPrediction', 'prediction', 'probability').show(20)

+-----+-------------+----------+-----------+
|Label|rawPrediction|prediction|probability|
+-----+-------------+----------+-----------+
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    0|   [17.0,3.0]|       0.0|[0.85,0.15]|
|    0|   [20.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    0|   [16.0,4.0]|       0.0|  [0.8,0.2]|
|    0|   [20.0,0.0]|       0.0|  [1.0,0.0]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    1|   [5.0,15.0]|       1.0|[0.25,0.75]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    1|   [6.0,14.0]|       1.0|  [0.3,0.7]|
|    1|   [3.0,17.0]|       1.0|[0.15,0.85]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    0|   [17.0,3.0]|       0.0|[0.85,0.15]|
|    1|   [5.0,15.0]|       1.0|[0.25,0.75]|
|    0|   [18.0,2.0]|       0.0|  [0.9,0.1]|
|    0|   [19.0,1.0]|       0.0|[0.95,0.05]|
|    0|   

In [58]:
### Train and test a Gradient Boosted Tree Classifier to make predicitions

from pyspark.ml.classification import GBTClassifier
gbtClassifier = GBTClassifier(featuresCol = 'features', labelCol = 'Label', maxDepth = 10, stepSize=.10)
gbtModel = gbtClassifier.fit(train)
predictions = gbtModel.transform(test)
predictions.select('Label', 'rawPrediction', 'prediction', 'probability').show(20)

+-----+--------------------+----------+--------------------+
|Label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    1|[-1.5435020027249...|       1.0|[0.04364652142729...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    0|[1.54350200272498...|       0.0|[0.95635347857270...|
|    1|[-1.5435020027249...|       1.0|[0.04364652142729...|
|    1|[-1.5435020027249

## Metrics Section for Evaluating Performance

In [ ]:
## write up Markdown on ROC etc.. by Steve to be inserted

In [72]:
#Compute Area under the Curve for Receiver Operating Characteristic and Precision Recall - the bigger the area the better the model.

from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
evaluator = BinaryClassificationEvaluator(labelCol='Label')

auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))

Area under ROC Curve: 1.0000
Area under PR Curve: 1.0000


In [ ]:
# Compute f1 score, precision, recall etc...  section - write up above it - Optional if we have time.  

Compute log loss - Steve will write up log loss explanation and hy we chose this

In [73]:
# Define a function clamp to restrict the values of probability to be greater than 0 and less than one
def clamp(n):
    epsilon = .000000000000001
    minn = 0 + epsilon
    maxn = 1 - epsilon
    return max(min(maxn, n), minn)

print(clamp (.5))

0.5


In [74]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# Define a UDF to extract the first element of the probability array returned which is probability of one
firstelement=udf(lambda v:clamp(float(v[1])))   #,FloatType() after [] was inserted and removed for epsilon
predictions.select(firstelement('probability')).show(5)

+---------------------+
|<lambda>(probability)|
+---------------------+
|                 0.05|
|                 0.15|
|              1.0E-15|
|                 0.05|
|                  0.2|
+---------------------+
only showing top 5 rows



In [75]:
# Create a new dataframe that contains a probability of one column (true)
import pyspark.sql.functions as f

predict_df = predictions.withColumn('prob_one', firstelement(predictions.probability))
predict_df.select('prob_one').show(10)
logloss = 0.0

+--------+
|prob_one|
+--------+
|    0.05|
|    0.15|
| 1.0E-15|
|    0.05|
|     0.2|
| 1.0E-15|
|    0.05|
|    0.75|
|    0.05|
|    0.05|
+--------+
only showing top 10 rows



In [76]:
# Compute the log loss for the spark dataframe for each row

row_logloss = (
    predict_df.withColumn(
        'logloss', -f.col('Label')*f.log(f.col('prob_one')) - (1.-f.col('Label'))*f.log(1.-f.col('prob_one'))
    )
)

logloss = row_logloss.agg(f.mean('logloss').alias('ll')).collect()[0]['ll']

In [77]:
# Examine the log loss
row_logloss.select('Label', f.round('logloss', 20), 'prediction', 'probability', 'prob_one').show(15)

+-----+-------------------+----------+-----------+--------+
|Label| round(logloss, 20)|prediction|probability|prob_one|
+-----+-------------------+----------+-----------+--------+
|    0|0.05129329438755058|       0.0|[0.95,0.05]|    0.05|
|    0|0.16251892949777494|       0.0|[0.85,0.15]|    0.15|
|    0|          9.992E-16|       0.0|  [1.0,0.0]| 1.0E-15|
|    0|0.05129329438755058|       0.0|[0.95,0.05]|    0.05|
|    0| 0.2231435513142097|       0.0|  [0.8,0.2]|     0.2|
|    0|          9.992E-16|       0.0|  [1.0,0.0]| 1.0E-15|
|    0|0.05129329438755058|       0.0|[0.95,0.05]|    0.05|
|    1| 0.2876820724517809|       1.0|[0.25,0.75]|    0.75|
|    0|0.05129329438755058|       0.0|[0.95,0.05]|    0.05|
|    0|0.05129329438755058|       0.0|[0.95,0.05]|    0.05|
|    0|0.05129329438755058|       0.0|[0.95,0.05]|    0.05|
|    0|0.05129329438755058|       0.0|[0.95,0.05]|    0.05|
|    1|0.35667494393873245|       1.0|  [0.3,0.7]|     0.7|
|    1|0.16251892949777494|       1.0|[0

In [78]:
# Model log loss total
print("log loss = ", logloss)

log loss =  0.1641667981794039


In [ ]:
####  APPENDIX - Misc Code I tried to be removed later

In [125]:
#import handyspark

In [119]:
predictions.select('probability').show(5)

+-----------+
|probability|
+-----------+
|  [1.0,0.0]|
|  [1.0,0.0]|
|  [1.0,0.0]|
|  [1.0,0.0]|
|  [1.0,0.0]|
+-----------+
only showing top 5 rows



In [ ]:
prob_zero = predictions.select(firstelement('probability'))
prob_zero.show(20)
prob_zero.printSchema()
#test.describe()

In [ ]:
prob_zero.select(prob_zero.columns[:1]).show(5)

In [ ]:
#dataframename.describe().show() - get ststatistics, min, max, ...

In [ ]:
prob_zero.select(prob_zero.columns[:1]).show(5)